# Assignment 4 for Course 1MS041
Make         sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline         and your highest score will be used.

---
## Assignment 4, PROBLEM 1
Maximum Points = 24


This time the assignment only consists of one problem, but we will do a more comprehensive analysis instead.

Consider the dataset `mammography.mat` that you can download from [http://odds.cs.stonybrook.edu/mammography-dataset/](http://odds.cs.stonybrook.edu/mammography-dataset/). Below you can find the code to load this file into `X` and `Y`, you just need to put the file in your `data` folder. During mammography the doctor would be looking for something called `calcification`, which is calcium deposits in the breast tissue and is used as an indicator of cancer. In this dataset the `X` corresponds to some measurements, there are 6 features. The `Y` is a 0-1 label where 1 represents calcification and 0 does not.

1. [3p] Split the data into three parts, train/test/validation where train is 60% of the data, test is 15% and validation is 25% of the data. Do not do this randomly, I have prepared a shuffling with a fixed seed, this way I can make sure that we all did the same splits. That is [train,test,validation] is the splitting layout.
2. [4p] Train a machine learning model on the training data (you are free to choose it yourself). Hint: you could always try `LogisticRegression`, but for it to work well you would need `StandardScaler` and put everything in a `Pipeline`.
3. [3p] Use the classification report from `Utils` and compute the intervals for precision-recall etc on the test set with `union_bound correction` with 95% confidence.
4. [3p] You are interested in minimizing the average cost of your classifier, but first we must define it:
    * If someone is calcified but classified as not, we say it costs 30 **(this is the worst scenario)** 
    * If someone is not calcified but classified as calcified, we say it costs 5 **(this probably only costs extra investigation)**
    * If someone is calcified but classified as calcified, costs 0 **(We did the right thing, no cost)**
    * If someone is not calcified but classified as not, costs 0 **(We did the right thing, no cost)**.

complete filling the function `cost` to compute the cost of a prediction model under a certain prediction threshold (recall our precision recall lecture and the `predict_proba` function from trained models). What would be the cost of having a model that always classifies someone as not calcified on the test set?

5. [4p] Now, we wish to select the threshold of our classifier that minimizes the cost, we do that by checking say 100 evenly spaced proposal thresholds between 0 and 1, and use our testing data to compute the cost.
6. [4p] With your newly computed threshold value, compute the cost of putting this model in production by computing the cost using the validation data. Also provide a confidence interval of the cost using Hoeffdings inequality with a 95% confidence.
7. [3p] Let $t$ be the threshold you found and $f$ the model you fitted, if we define the random variable
$$
    C = 30(1-1_{f(X)\geq t})Y+5(1-Y)1_{f(X) \geq t}
$$
then $C$ denotes the cost of a randomly chosen patient. In the above we are estimating $\mathbb{E}[C]$ using the empirical mean. However, since the number of calcifications in the population is fairly small and our classifier probably has a fairly high precision for the $0$ class, then $C$ should have fairly small variance. Compute the empirical variance of $C$ on the validation set. What would be the confidence interval if we used Bennett's inequality instead of Hoeffding in point 6 but with the computed empirical variance as our guess for the variance?

In [12]:
import scipy.io as so
import numpy as np
data = so.loadmat('data/mammography.mat')

np.random.seed(0)
shuffle_index = np.arange(0,len(data['X']))
np.random.shuffle(shuffle_index)

X = data['X'][shuffle_index,:]
Y = data['y'][shuffle_index,:].flatten()

In [13]:

# Part 1
from sklearn.model_selection import train_test_split
# Part 1
# Split the X,Y into three parts, make sure that the sizes are
# (6709, 6), (1677, 6), (2797, 6), (6709,), (1677,), (2797,)
# Splitting data into train+test (75%) and validation (25%)
total_size = 11183
train_size = (6709+1677)/11183
test_size = 6709 / (6709+1677)
X_train_test, X_valid, Y_train_test, Y_valid = train_test_split(X, Y, train_size=train_size, shuffle=False)

# Splitting train+test into train (60% of 75%) and test (15% of 75%)
X_train, X_test, Y_train, Y_test = train_test_split(X_train_test, Y_train_test, train_size = test_size,shuffle = False)  
# Split the X,Y into three parts, make sure that the sizes are
# (6709, 6), (1677, 6), (2797, 6), (6709,), (1677,), (2797,)


In [14]:

# Part 2
# Use X_train,Y_train to train a model from sklearn. Make sure it has the predict_proba function
# for instance LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

model = Pipeline([
    ('scaler', StandardScaler()),
    ('log_reg', LogisticRegression())
])

model.fit(X_train, Y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('log_reg', LogisticRegression())])

In [15]:

# Part 3

# Compute precision and recall on the test set using 
from Utils import classification_report_interval

# Each precision and recall should be a tuple, for instance you can write
# precision0 = (0.9,0.95)
# Generate predictions on the test set
y_pred_proba = model.predict(X_test)

# You might need to adjust the function call according to its actual signature
report = classification_report_interval(y_pred=y_pred_proba, y_true=Y_test,labels = [0,1], alpha=0.05)
# Extracting lines from the report
lines = report.split('\n')

# Assuming the format is consistent and intervals are on lines 2 and 3
# Format: label precision_interval recall_interval
precision0_line = lines[2].split()
precision1_line = lines[3].split()
# Print the lines for debugging
#print("Precision0 line:", precision0_line)
#print("Precision1 line:", precision1_line)
# Extracting intervals
# The intervals are in the format "number : [interval_start,interval_end]"
import re
# Corrected function to extract intervals
def extract_interval(s):
    return tuple(map(float, s.strip('[]').split(',')))

# Corrected extraction of intervals for class 0
precision0 = extract_interval(precision0_line[3])  # Precision interval for class 0
recall0 = extract_interval(precision0_line[6])     # Recall interval for class 0

# Corrected extraction of intervals for class 1
precision1 = extract_interval(precision1_line[3])  # Precision interval for class 1
recall1 = extract_interval(precision1_line[6])     # Recall interval for class 1


# The code below will check that you supply the proper type
assert(type(precision0) == tuple)
assert(len(precision0) == 2)
assert(type(recall0) == tuple)
assert(len(recall0) == 2)
assert(type(precision1) == tuple)
assert(len(precision1) == 2)
assert(type(recall1) == tuple)
assert(len(recall1) == 2)

In [16]:

# Part 4
def cost(model, threshold, X, Y):
    pred_proba = model.predict_proba(X)[:, 1]
    predictions = (pred_proba >= threshold)
    false_negatives = (Y == 1) & (predictions == 0)
    false_positives = (Y == 0) & (predictions == 1)
    cost = (30 * false_negatives.sum() + 5 * false_positives.sum()) / len(Y)
    return cost

In [17]:

# Part 4

# Fill in the naive cost of the model that would classify all as non-calcified on the test set

naive_cost = cost(model, 1, X_test, Y_test)

In [18]:

# Part 5

thresholds = np.linspace(0, 1, 100)
costs = [cost(model, t, X_test, Y_test) for t in thresholds]
optimal_index = np.argmin(costs)
optimal_threshold = thresholds[optimal_index]
cost_at_optimal_threshold = costs[optimal_index]

In [19]:

# Part 6

import numpy as np

def hoeffding_interval(sample_mean, n, a, b, confidence_level=0.95):
    epsilon = np.sqrt(np.log(2 / (1 - confidence_level)) / (2 * n))
    return max(a, sample_mean - epsilon * (b - a)), min(b, sample_mean + epsilon * (b - a))

# Compute the average cost on the validation set
avg_cost_validation = cost(model, optimal_threshold, X_valid, Y_valid)

# Use Hoeffding's inequality to compute the confidence interval
n = len(Y_valid)  # Number of samples in the validation set
a, b = 0, 30  # Bounds of the cost
confidence_level = 0.95  # Confidence level

cost_interval = hoeffding_interval(avg_cost_validation, n, a, b, confidence_level)

assert(type(cost_interval) == tuple)
assert(len(cost_interval) == 2)


In [20]:

# Part 7

C = 30 * (1 - (model.predict_proba(X_valid)[:, 1] >= optimal_threshold)) * Y_valid + 5 * (1 - Y_valid) * (model.predict_proba(X_valid)[:, 1] >= optimal_threshold)
variance_of_C = np.var(C)

In [21]:

# Part 7

# A 95% confidence interval of the random variable C using Bennett's inequality
#interval_of_C = XXX
def bennett_interval(sample_mean, variance, n, b, confidence_level=0.95):
    epsilon = np.sqrt(2 * variance * np.log(2 / (1 - confidence_level)) / n + 2 * b * np.log(2 / (1 - confidence_level)) / (3 * n))
    return sample_mean - epsilon, sample_mean + epsilon

# Mean and variance of C
mean_of_C = avg_cost_validation  # From Part 6
variance_of_C = np.var(C)  # You should have this from a previous step

# Maximum deviation (b)
b = 30  # Maximum cost

# Use Bennett's inequality to compute the confidence interval
interval_of_C = bennett_interval(mean_of_C, variance_of_C, len(Y_valid), b, confidence_level=0.95)

assert(type(interval_of_C) == tuple)
assert(len(interval_of_C) == 2)